In [155]:
import psycopg2
import time
import sys
   
conn = psycopg2.connect(database="tcount", user="postgres")
cur = conn.cursor()

sql = "delete from Tweetwordcount ;" 
cur.execute(sql)
conn.commit()
conn.close()

In [178]:
conn = psycopg2.connect(database="tcount", user="postgres")
cur = conn.cursor()

sql = "select count(*) from Tweetwordcount ;" 
cur.execute(sql)
print cur.fetchall()
conn.commit()
conn.close()

[(20079L,)]


In [166]:
conn = psycopg2.connect(database="tcount", user="postgres")
cur = conn.cursor()

word = 'andreasoto'
sql = "SELECT * FROM Tweetwordcount WHERE word='%s';" %(word)
cur.execute(sql)
test = cur.fetchall()
conn.commit()
conn.close()

In [167]:
len(test)

0

---
# Serving Scripts

## finalresults.py

This script gets a word as an argument and returns the total number of word occurrences in the stream. This script can also be used to query all the word counts, order them alphabetically, and report the first N word counts. The default N is 25, but this number can be sent as an input.

**USAGE**

To get the number of occurances of a single word:  
> `python finalresults.py hello`
    
Get all the word counts, sorted alphabetically, one per line:
> `python finalresults.py`
    
The default is to show only the first 25 words. To increase the number of words shown, send the number of records to show as an input:
> `python finalresults.py 2000`

In [171]:
%%writefile analysis/finalresults.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2
import time
import sys
   
def get_wordCount(word, show=25):
    # Get count for the input word or all word counts. 
    # The output is restricted to the 'show' value.
    
    # Connect to database
    conn = psycopg2.connect(database="tcount", user="postgres")
    cur = conn.cursor()
    
    # Get total words counted in tweets
    sql = "SELECT count(*) FROM Tweetwordcount ;" 
    cur.execute(sql)
    total_words = cur.fetchall()
    conn.commit()
    
    # Get count for word or all words (depending on input)
    if word == None:
        sql = "SELECT * FROM Tweetwordcount ORDER BY word ASC LIMIT %d;"%(min(show,total_words[0][0]))        
    else:
        sql = "SELECT * FROM Tweetwordcount WHERE word='%s';" %(word)
    
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    conn.close()
    
    # Print results
    if word == None:
        print 'First %d word-counts (out of %d words):\n'%(min(show,total_words[0][0]),total_words[0][0])
        print "%12s  %s"%('Word','Count')
        for w,c in result:
            print "%12s  %d"%(w,c)
    else:
        print "Number of occurences of '%s':  %d"%(result[0][0], result[0][1]), "\t@", time.ctime(time.time())

# ===================================================================================
if __name__ == '__main__':
    '''
    To get the number of occurances of single word:
       python finalresults.py hello
    
    Get all the word counts, sorted alphabetically, one per line:
       python finalresults.py
    
    The default is to show only the first 25 words. 
    To increase the number of words shown, send any number as an input:
       python finalresults.py 2000
    '''    
    
    numToShow = 25
    word = None
    # Get input word if any
    if len(sys.argv) > 1:
        # Get target word
        if sys.argv[1].isdigit():
            numToShow = int(sys.argv[1])
        else:
            word = sys.argv[1]
    
    get_wordCount(word, numToShow)

Overwriting analysis/finalresults.py


### Sample results

In [158]:
!python analysis/finalresults.py 10

First 10 word-counts (out of 777 words):

        Word  Count
           a  63
        able  2
       about  7
   according  1
     account  2
    actually  3
     address  3
       adele  1
      adidas  2
       after  1


In [159]:
!python analysis/finalresults.py the

Number of occurences of 'the':  174 	@ Thu Dec  3 01:23:13 2015


In [174]:
!python analysis/finalresults.py the

Number of occurences of 'the':  20624 	@ Thu Dec  3 02:14:29 2015


In [177]:
!python analysis/finalresults.py the

Number of occurences of 'the':  49 	@ Thu Dec  3 02:15:14 2015


In [164]:
!python finalresults.py 10

First 10 word-counts (out of 5294 words):

        Word  Count
           a  1392
         aaa  2
       aaaaa  2
       aaand  2
         aap  2
         aba  2
         abc  2
   abductors  2
     abiding  2
     ability  4


## histogram.py

In [ ]:
%%writefile analysis/histogram.py
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2
import time
import sys
   
def get_wordCount(word, show=25):
    # Get count for the input word or all word counts. 
    # The output is restricted to the 'show' value.
    
    # Connect to database
    conn = psycopg2.connect(database="tcount", user="postgres")
    cur = conn.cursor()
    
    # Get total words counted in tweets
    sql = "SELECT count(*) FROM Tweetwordcount ;" 
    cur.execute(sql)
    total_words = cur.fetchall()
    conn.commit()
    
    # Get count for word or all words (depending on input)
    if word == None:
        sql = "SELECT * FROM Tweetwordcount ORDER BY word ASC LIMIT %d;"%(min(show,total_words[0][0]))        
    else:
        sql = "SELECT * FROM Tweetwordcount WHERE word='%s';" %(word)
    
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    conn.close()
    
    # Print results
    if word == None:
        print 'First %d word-counts (out of %d words):\n'%(min(show,total_words[0][0]),total_words[0][0])
        print "%12s  %s"%('Word','Count')
        for w,c in result:
            print "%12s  %d"%(w,c)
    else:
        print "Number of occurences of '%s':  %d"%(result[0][0], result[0][1]), "\t@", time.ctime(time.time())

# ===================================================================================
if __name__ == '__main__':
    '''
    To get the number of occurances of single word:
       python finalresults.py hello
    
    Get all the word counts, sorted alphabetically, one per line:
       python finalresults.py
    
    The default is to show only the first 25 words. 
    To increase the number of words shown, send any number as an input:
       python finalresults.py 2000
    '''    
    
    numToShow = 25
    word = None
    # Get input word if any
    if len(sys.argv) > 1:
        # Get target word
        if sys.argv[1].isdigit():
            numToShow = int(sys.argv[1])
        else:
            word = sys.argv[1]
    
    get_wordCount(word, numToShow)

---
# Spout: tweet.py

In [12]:
%%writefile src/spouts/tweets.py
from __future__ import absolute_import, print_function, unicode_literals
#TEST THIS
import itertools, time
import tweepy, copy 
import Queue, threading

from streamparse.spout import Spout

################################################################################
# Twitter credentials
################################################################################
twitter_credentials = {
    "consumer_key"        :  "lxm4Nf8lJElAi5VCvbCTAeSj1",
    "consumer_secret"     :  "KKWw60hoRLU3TOkiHtT2xLOTu3G3bcQ6eee9kbZPahfxVDiB3F",
    "access_token"        :  "195097748-N9FbGLe2H32RPzMy9upSbflkk7GisamQ0XFt76Rg",
    "access_token_secret" :  "4aiLQlyUkeFnJ2Dhw5FS9TZn2oHLdqsivH7YKXRwnBkGj"
}

def auth_get(auth_key):
    if auth_key in twitter_credentials:
        return twitter_credentials[auth_key]
    return None

################################################################################
# Class to listen and act on the incoming tweets
################################################################################
class TweetStreamListener(tweepy.StreamListener):

    def __init__(self, listener):
        self.listener = listener
        super(self.__class__, self).__init__(listener.tweepy_api())

    def on_status(self, status):
        self.listener.queue().put(status.text, timeout = 0.01)
        return True
  
    def on_error(self, status_code):
        return True # keep stream alive
  
    def on_limit(self, track):
        return True # keep stream alive

class Tweets(Spout):

    def initialize(self, stormconf, context):
        self._queue = Queue.Queue(maxsize = 100)

        consumer_key = auth_get("consumer_key") 
        consumer_secret = auth_get("consumer_secret") 
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

        if auth_get("access_token") and auth_get("access_token_secret"):
            access_token = auth_get("access_token")
            access_token_secret = auth_get("access_token_secret")
            auth.set_access_token(access_token, access_token_secret)

        self._tweepy_api = tweepy.API(auth)

        # Create the listener for twitter stream
        listener = TweetStreamListener(self)

        # Create the stream and listen for english tweets
        stream = tweepy.Stream(auth, listener, timeout=None)
        stream.filter(languages=["en"], track=["a", "the", "i", "you", "u"], async=True)

    def queue(self):
        return self._queue

    def tweepy_api(self):
        return self._tweepy_api

    def next_tuple(self):
        try:
            tweet = self.queue().get(timeout = 0.1) 
            if tweet:
                self.queue().task_done()
                self.emit([tweet])
 
        except Queue.Empty:
            self.log("Empty queue exception")
            time.sleep(0.1) 

    def ack(self, tup_id):
        pass  # if a tuple is processed properly, do nothing

    def fail(self, tup_id):
        pass  # if a tuple fails to process, do nothing

Overwriting src/spouts/tweets.py


# Bolt: parse.py

In [157]:
%%writefile src/bolts/parse.py 
from __future__ import absolute_import, print_function, unicode_literals

import re
from streamparse.bolt import Bolt

################################################################################
# Function to check if the string contains only ascii chars
################################################################################
def ascii_string(s):
    return all(ord(c) < 128 for c in s)

class ParseTweet(Bolt):

    def process(self, tup):
        tweet = tup.values[0]  # extract the tweet

        # Split the tweet into words
        words = tweet.split()

        # Filter out the hash tags, RT, @ and urls
        valid_words = []
        for word in words:

            # Filter the hash tags
            if word.startswith("#"): continue

            # Filter the user mentions
            if word.startswith("@"): continue

            # Filter out retweet tags
            if word.startswith("RT"): continue

            # Filter out the urls
            if word.startswith("http"): continue

            # Strip leading and lagging punctuations
            aword = word.strip("\"?><,'.:;)")
            # Clean other charactes from string 
            aword = aword.lower()
            # Basic word cleaning
            aword = re.sub("'","",aword)
            aword = re.sub("/","", aword)
            aword = re.sub("\)","", aword)
            aword = re.sub("\(","", aword)
            aword = re.sub("[0-9!@#$%^&*-_+=~{}|:;<>?,.]","", aword)
            aword = aword.replace("\\","")

            # now check if the word contains only ascii
            if len(aword) > 0 and ascii_string(word):
                valid_words.append([aword])

        if not valid_words: return

        # Emit all the words
        self.emit_many(valid_words)

        # tuple acknowledgement is handled automatically

Overwriting src/bolts/parse.py


# Bolt: wordcount.py

In [169]:
%%writefile src/bolts/wordcount.py
from __future__ import absolute_import, print_function, unicode_literals

from collections import Counter
from redis import StrictRedis
from streamparse.bolt import Bolt

import psycopg2
import time

class WordCounter(Bolt):

    def initialize(self, conf, ctx):
        self.counts = Counter()
        self.redis = StrictRedis()

    def process(self, tup):
        word = tup.values[0]

        # Write codes to increment the word count in Postgres
        # Use psycopg to interact with Postgres
        # Database name: Tcount 
        # Table name: Tweetwordcount 
        
        # Connect to database
        conn = psycopg2.connect(database="tcount", user="postgres")
        cur = conn.cursor()
        
        # Increment the local count
        self.counts[word] += 1
        
        # Check if word is already in the table 
        # (to stop and restart stream process without overwriting current table or deleting it)
        sql = "SELECT * FROM Tweetwordcount WHERE word='%s';" %(word)
        cur.execute(sql)
        has_word = cur.fetchall()
        conn.commit()
        
        if len(has_word) == 0:
            # New word > INSERT INTO
            sql = "INSERT INTO Tweetwordcount (word,count) VALUES ('%s', %d);" %(unicode(word), self.counts[word])
        else:
            # Update word count > UPDATE
            sql = "UPDATE Tweetwordcount SET count=%d WHERE word='%s';" %(self.counts[word], unicode(word))

        cur.execute(sql)
        conn.commit()
        #self.emit([word, self.counts[word]])

        # Log the count - just to see the topology running
        self.log('%s: %d' % (word, self.counts[word]))

Overwriting src/bolts/wordcount.py
